# Create 3D Models with Nerf Studio from iPad with Camera

Author: Deeepwin  
Date: 26.07.2023 
***

## Installation

Run this command in a terminal to create new conda environment:     

`conda create --name nerfstudio python=3.8`

### Install CUDA and PyTorch

In [ ]:
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118

Install NVCC in conda, we will need it for tiny-cuda-nn.

In [ ]:
!conda install -y -c "nvidia/label/cuda-11.8.0" cuda-toolkit
!conda install -y -c "nvidia/label/cuda-11.8.0" cuda-nvcc

In [ ]:
# check CUDA versions, must be 11.8
!conda list cudatoolkit
!conda list cudnn
!conda list cuda-nvcc

### Setup Tiny CUDA NN

Now set CUDA_HOME enviroment variable. Otherwise you cannot install `tiny-cuda-nn`. Update path according to your Conda installation.

In [ ]:
!conda env config vars set CUDA_HOME="/home/martin/anaconda3/envs/nerfstudio/"
!conda env config vars list

For CUDA version 11.8 you need g++ version 9 (max version is 11), see [here](https://stackoverflow.com/questions/6622454/cuda-incompatible-with-my-gcc-version). General instructions in how to setup different versions of g++ can be found [here](https://tuxamito.com/wiki/index.php/Installing_newer_GCC_versions_in_Ubuntu).

In [ ]:
from getpass import getpass
p = getpass()
!echo {p} | sudo -S apt -y install g++-9
!echo {p} | sudo -S update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-9 9
!echo {p} | sudo -S update-alternatives --config g++
!echo {p} | sudo -S apt -y install gcc-9
!echo {p} | sudo -S update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-9 9
!echo {p} | sudo -S update-alternatives --config gcc
del p

In [ ]:
!g++ --version
!gcc --version

Now install Tiny CUDA.

In [ ]:
!pip install ninja git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

In [ ]:
# check Pytorch version, must be 2.0.1 and CUDA 11.8
torch_version = !pip show torch
torch_version[1]

Install Nerfstudio see [website](https://docs.nerf.studio/en/latest/quickstart/installation.html).

In [ ]:
# install latest to ensure compatibility, run in separate terminal in case of error
!git clone https://github.com/nerfstudio-project/nerfstudio.git
!cd nerfstudio
!pip install --upgrade pip setuptools
!pip install -e .
!cd ..

## Train Model from Nerfstudio

In [ ]:
# download data from nerfstudio
!ns-download-data nerfstudio --capture-name=poster

In [ ]:
# train from scratch
!ns-train nerfacto --data data/nerfstudio/poster

In [ ]:
# export as point cloud, adjust export directory
%%capture
!ns-export pointcloud \
    --load-config outputs/poster/nerfacto/2023-03-18_122407/config.yml \
    --output-dir exports/pcd/ \
    --num-points 1000000 \
    --remove-outliers True \ 
    --estimate-normals False \ 
    --use-bounding-box True \
    --bounding-box-min -1 -1 -1 \ 
    --bounding-box-max 1 1 1 \

In [ ]:
#  open in separate window
%%capture
!open3d draw exports/pcd/point_cloud.ply 

An example view of the point cloud data.

<img src="pics/chair.jpg" height="200"/>


## Train your Own Model

If you have no phone with LiDAR like me, install [Kiri Engine](https://www.kiriengine.com/) app. Follow the instructions [here](https://docs.nerf.studio/en/latest/quickstart/custom_dataset.html). 

1. Once engine has processed data, export by sending link to file by email.
2. Extract file and place in `export` folder with proper name.
3. Adjust `--data` path string below. Change websocket port in case you get an error.

In [ ]:
import os

# give experiment a name
data_name = 'mouse-2'

# model
model_name = 'mipnerf'

# define custom data folder location
data_input_dir = os.path.join('data', 'mine', data_name)

In [15]:
!ns-train $model_name \
    --vis tensorboard \
    --viewer.websocket-port 7007 \
    --experiment-name $data_name \
    nerfstudio-data --data $input_dir

Step (% Done)       Train Iter (time)    ETA (time)           Train Rays / Sec     Test Rays / Sec       
-------------------------------------------------------------------------------------------------------- 
870 (0.09%)         136.075 ms           1 d, 13 h, 45 m, 56 s 7.83 K                                    
880 (0.09%)         136.872 ms           1 d, 13 h, 59 m, 11 s 7.79 K                                    
890 (0.09%)         136.142 ms           1 d, 13 h, 47 m, 1 s 7.82 K                                     
900 (0.09%)         135.359 ms           1 d, 13 h, 33 m, 56 s 7.87 K                                    
910 (0.09%)         137.873 ms           1 d, 14 h, 15 m, 47 s 7.74 K                                    
920 (0.09%)         139.824 ms           1 d, 14 h, 48 m, 15 s 7.63 K                                    
930 (0.09%)         138.753 ms           1 d, 14 h, 30 m, 24 s 7.67 K                                    
940 (0.09%)         138.680 ms           1 d, 

Open viewer and monitor progress. Once training is complete, look up output folder an adjust following variables:

In [ ]:
experiment_id = '2023-07-30_193013'

In [ ]:
output_dir = os.path.join('outputs', data_name, model_name, experiment_id)

In [ ]:
!ns-export pointcloud \
    --load-config $output_dir/config.yml \
    --output-dir exports/$data_name \
    --num-points 100000 \
    --remove-outliers True \
    --use-bounding-box True \
    --bounding-box-min -1 -1 -1 \
    --bounding-box-max 1 1 1 \
    --num-rays-per-batch 4096 \
    --rgb_output_name rgb_fine \
    --depth_output_name rgb_fine

In [ ]:
# look at point cloud
!open3d draw exports/$data_name/point_cloud.ply

Here an example of a plant on a table.


<table><tr>
<td> <img src="pics/plant-0.jpg" height="300"/>  </td>
<td> <img src="pics/plant-1.jpg" height="300"/>  </td>
<td> <img src="pics/plant-2.jpg" height="300"/>  </td>
</tr></table>


Here an example of my Logitech vertical mouse to see the geometric accuracy.

<table><tr>
<td> <img src="pics/mouse-2-0.jpg" height="300"/>  </td>
<td> <img src="pics/mouse-2-1.jpg" height="300"/>  </td>
<td> <img src="pics/mouse-2-2.jpg" height="300"/>  </td>
<tr>
<td> <img src="pics/mouse-2-3.jpg" height="300"/>  </td>
<td> <img src="pics/mouse-2-4.jpg" height="300"/>  </td>
<td> <img src="pics/mouse-2-5.jpg" height="300"/>  </td>
</tr>
</tr></table>

Another example with reflective surfaces on a kettle.

<table><tr>
<td> <img src="pics/kettle-2-0.jpg" height="300"/>  </td>
<td> <img src="pics/kettle-2-1.jpg" height="300"/>  </td>
<td> <img src="pics/kettle-2-2.jpg" height="300"/>  </td>
<td> <img src="pics/kettle-2-3.jpg" height="300"/>  </td>
</tr></table>


## Nerfstudio Viewer

In [ ]:
!ns-viewer --load-config outputs/$object_name/nerfacto/$training_id/config.yml

## Extract Mesh

In [ ]:
!ns-export tsdf --load-config outputs/$object_name/nerfacto/$training_id/config.yml --output-dir $output_dir

In [ ]:
!ns-export poisson --load-config outputs/$object_name/nerfacto/$training_id/config.yml --output-dir $output_dir

In [ ]:
!open3d draw $output_dir/poisson_mesh.ply 